In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [2]:
def get_page(url):
    response = requests.get(url)    #make the HTTP requests
    try:
        if response.status_code == 200:  #code 200 indicates a successful request
            return response              #return the response content
        else:
            return None
    except RequestException as e:       #if the request is not successful, print out the exceptions content
        print('Requests Failed: '+str(e))

In [3]:
main_page = get_page("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population")

ConnectionError: HTTPSConnectionPool(host='en.wikipedia.org', port=443): Max retries exceeded with url: /wiki/List_of_United_States_cities_by_population (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000020A7913D100>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [4]:
#other parsers like 'html.parser' and 'xml' can also be considered
soup = BeautifulSoup(main_page.text,'lxml')  

In [5]:
#find the table element
cities = soup.find('table', attrs={'class':'wikitable sortable'})   

In [6]:
#read the html content into strings
df = pd.read_html(str(cities))  

#build data frame
newdf = df[0]                  

#retrieve the column names of the data frame from the first row
columns = newdf.iloc[0].tolist(); columns 

[1,
 'New York City[d]',
 'New York',
 8336817,
 8175133,
 '+1.98%',
 '301.5\xa0sq\xa0mi',
 '780.9\xa0km2',
 '28,317/sq\xa0mi',
 '10,933/km2',
 '.mw-parser-output .geo-default,.mw-parser-output .geo-dms,.mw-parser-output .geo-dec{display:inline}.mw-parser-output .geo-nondefault,.mw-parser-output .geo-multi-punct{display:none}.mw-parser-output .longitude,.mw-parser-output .latitude{white-space:nowrap}40°39′49″N 73°56′19″W\ufeff / \ufeff40.6635°N 73.9387°W']

In [7]:
#remove duplicated columns
newdf = newdf.drop([7,9],axis=1)       

colname = ['2018 Rank','City','State','2018 Population Estimate','2010 Population Estimate','Population Change','2016 Land Area','2016 Population Density','Location']

#rename the columns of the data frame for easier interpretation
newdf.columns = colname    

#remove the first row
newdf = newdf.drop([0],axis=0)

KeyError: '[7 9] not found in axis'

In [9]:
for i in range(0,len(newdf)):
    
    #remove the annotation symbol
    newdf.iloc[i,1] = re.sub('\[.*?\]','',newdf.iloc[i,1])    
newdf

,2019rank,City,State[c],2019estimate,2010Census,Change,2016 land area,2016 land area.1,2016 population density,2016 population density.1,Location
0,1,New York City,New York,8336817,8175133,+1.98%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",".mw-parser-output .geo-default,.mw-parser-outp..."
1,2,Los Angeles,California,3979576,3792621,+4.93%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°W
2,3,Chicago,Illinois,2693976,2695598,−0.06%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W
3,4,Houston,Texas,2320268,2100263,+10.48%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W
4,5,Phoenix,Arizona,1680992,1445632,+16.28%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°W
...,...,...,...,...,...,...,...,...,...,...,...
312,313,San Angelo,Texas,101004,93200,+8.37%,59.9 sq mi,155.1 km2,"1,681/sq mi",649/km2,31°26′28″N 100°27′02″W﻿ / ﻿31.4411°N 100.4505°W
313,314,Vacaville,California,100670,92428,+8.92%,29.0 sq mi,75.1 km2,"3,449/sq mi","1,332/km2",38°21′14″N 121°58′22″W﻿ / ﻿38.3539°N 121.9728°W
314,315,Clinton,Michigan,100471,96796,+3.80%,28.1 sq mi,72.8 km2,"3,573/sq mi","1,380/km2",42°35′25″N 82°55′01″W﻿ / ﻿42.5903°N 82.9170°W
315,316,Bend,Oregon,100421,76639,+31.03%,33.1 sq mi,85.7 km2,"3,034/sq mi","1,171/km2",44°03′00″N 121°18′00″W﻿ / ﻿44.0500°N 121.3000°W


In [10]:
citynames = newdf['City'].tolist()    #find all the top cities' names
links = cities.find_all('a')      #find all the a elements in the table
wikilinks = []                  #create a new list to contain the links of wikipedia links

for link in links:         #for each a element
    if link.text in citynames:     #if the text of the link matches any of the city names
        wikilinks.append("https://en.wikipedia.org"+link.get('href'))  #concatenate the link with the wikipedia's url and add them to the wikilinks list

In [11]:
num = 0
for link in wikilinks:
    cityres = get_page(link)      #for each link in the wikilinks list, make HTTP requests to the web page
    
    citysoup = BeautifulSoup(cityres.text,'lxml')   #parse each HTML file using BeautifulSoup
    info = citysoup.find('table',attrs={"class":"infobox"})  #find the information box in each HTML file
    
    df = pd.read_html(str(info))   #read the content of the information box into strings
    clist = df[0]
    
    ind = clist[clist[0]=='Country'].index[0]   
    clist = clist.drop(clist.index[range(0,ind+1)])   #remove the redundant information of the city's pic & map
   
    for i in range(0,len(clist)):         #for each row of clist where the 1st column contains attributes name and 2nd column contains values
        if type(clist.iloc[i,0])==str:    #if the attribute name is string (not NA or null)
            clist.iloc[i,0] = re.sub('^•\s|\[.*?\]','',clist.iloc[i,0])  #remove redundant symbols
            clist.iloc[i,0] = re.sub('[\(\)]','',clist.iloc[i,0])
            if len(clist.iloc[i,0].split())<=4 and clist.iloc[i,0] not in colname:   #if the attribute name is less than 5 words and does not have duplicates in the original dataset
                clist.iloc[i,0] = clist.iloc[i,0].lower()       #change the attribute name into lowercase letters to aviod duplications
                if clist.iloc[i,0] not in newdf.columns:    #if the attribute name is not already a column name in the original data frame
                    newdf[clist.iloc[i,0]] = ""           #add a new column to the original data frame with the attribute name and set the default value as blank strings
                index = newdf.columns.get_loc(clist.iloc[i,0])    #find the index of the column that the new data should belong to
                newdf.iloc[num,index]=clist.iloc[i,1]         #add the new data into corresponding columns in the orginal data frame
    num=num+1     #store the index of the city


KeyError: 0

In [12]:
topcities = newdf.copy()    #make a copy of the newdf

In [13]:
def combine(origin,new):  #define a function to insert the data from the 'new' column into the 'origin' column
    for i in topcities[topcities[new]!=""].index:  #for each row of the 'new' column which contains non-null value
        topcities.iloc[i-1,topcities.columns.get_loc(origin)] = topcities.iloc[i-1,topcities.columns.get_loc(new)] #insert the value in the row into the 'origin' column

combine('zip codes','zip code')  #call the function to modify the columns
combine('area codes','area code')
combine('demonyms','demonym')
combine('county','counties')

for column in topcities.columns:        #for each column in the topcities data frame
    if sum(topcities[column]!="")< 100:   #if the columns contains less than 100 non-null values
        topcities = topcities.drop(column,axis=1)   #remove the column

KeyError: 'zip code'

In [14]:
topcities.columns = ['Rank2018', 'City', 'State', 'Population Estimate2018',
       'Population Estimate2010', 'Population Change Percentage',
       'Land Area2016', 'Population Density2016',
       'Location', 'Named For', 'government', 'Government Type', 'Mayor', 'area', 'total',
       'Land Area Latest', 'Water Area Latest', 'Metro Area Latest', 'Elevation', 'population 2010', 'rank',
       'density', 'Demonyms', 'Time Zone', 'Summer DST', 'Zip Codes',
       'Area Codes', 'FIPS Code', 'GNIS Feature Id', 'Website', 'County',
       'Incorporated', 'estimate 2017', 'Urban Area Latest', 'City Manager']      #modify the column names for easier interpretation

uselessvar = ['government', 'area','total','population 2010', 'rank',
       'density', 'estimate 2017']      #list of useless column names
for var in uselessvar:   #for each column in the useless columns
    index = topcities.columns.get_loc(var)    #find the index of the column
    topcities = topcities.drop(topcities.columns[index],axis=1)    #remove the column from the data frame


ValueError: Length mismatch: Expected axis has 11 elements, new values have 35 elements

In [15]:
topcities['Description'] = ''  #add a new column Description and set the default value as blank strings
index = topcities.columns.get_loc('Description')   #store the index of the Description column in a variable
num = 0
for link in wikilinks:      #for each city's wikipedia link
    cityres = get_page(link)    #make HTTP requests
    citysoup = BeautifulSoup(cityres.text,'lxml')   #parse the HTML files
    desp = citysoup.find('div',attrs={"class":"mw-parser-output"})   #find the div element
    df = desp.find_all('p',attrs={'class': None})  #find all the p elements in div
    description = re.sub('\[.*?\]|\\n|\\xa0|\s\(listen\)','', df[0].text)     #find the text of the first paragraph and remove meaningless symbols
    topcities.iloc[num,index] = description     #add the text into the new column
    num+=1    #document the index of the city

IndexError: single positional indexer is out-of-bounds

In [16]:
topcities

,2019rank,City,State[c],2019estimate,2010Census,Change,2016 land area,2016 land area.1,2016 population density,2016 population density.1,Location,Description
0,1,New York City,New York,8336817,8175133,+1.98%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",".mw-parser-output .geo-default,.mw-parser-outp...","New York City (NYC), often called simply New Y..."
1,2,Los Angeles,California,3979576,3792621,+4.93%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°W,Los Angeles (/lɔːs ˈændʒələs/; Spanish: Los Án...
2,3,Chicago,Illinois,2693976,2695598,−0.06%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W,"Chicago (/ʃɪˈkɑːɡoʊ/ shih-KAH-goh, locally als..."
3,4,Houston,Texas,2320268,2100263,+10.48%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W,Houston (/ˈhjuːstən/ HEW-stən) is the most pop...
4,5,Phoenix,Arizona,1680992,1445632,+16.28%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°W,Phoenix (/ˈfiːnɪks/ FEE-niks; Navajo: Hoozdo; ...
...,...,...,...,...,...,...,...,...,...,...,...,...
312,313,San Angelo,Texas,101004,93200,+8.37%,59.9 sq mi,155.1 km2,"1,681/sq mi",649/km2,31°26′28″N 100°27′02″W﻿ / ﻿31.4411°N 100.4505°W,"Spokane Valley is a city in Spokane County, Wa..."
313,314,Vacaville,California,100670,92428,+8.92%,29.0 sq mi,75.1 km2,"3,449/sq mi","1,332/km2",38°21′14″N 121°58′22″W﻿ / ﻿38.3539°N 121.9728°W,San Angelo /sæn ˈændʒəloʊ/ is a city in and th...
314,315,Clinton,Michigan,100471,96796,+3.80%,28.1 sq mi,72.8 km2,"3,573/sq mi","1,380/km2",42°35′25″N 82°55′01″W﻿ / ﻿42.5903°N 82.9170°W,Vacaville is a city located in Solano County i...
315,316,Bend,Oregon,100421,76639,+31.03%,33.1 sq mi,85.7 km2,"3,034/sq mi","1,171/km2",44°03′00″N 121°18′00″W﻿ / ﻿44.0500°N 121.3000°W,"Clinton Township, officially the Charter Towns..."


In [17]:
topcities.head()  #inspect the first few rows in topcities

,2019rank,City,State[c],2019estimate,2010Census,Change,2016 land area,2016 land area.1,2016 population density,2016 population density.1,Location,Description
0,1,New York City,New York,8336817,8175133,+1.98%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",".mw-parser-output .geo-default,.mw-parser-outp...","New York City (NYC), often called simply New Y..."
1,2,Los Angeles,California,3979576,3792621,+4.93%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°W,Los Angeles (/lɔːs ˈændʒələs/; Spanish: Los Án...
2,3,Chicago,Illinois,2693976,2695598,−0.06%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W,"Chicago (/ʃɪˈkɑːɡoʊ/ shih-KAH-goh, locally als..."
3,4,Houston,Texas,2320268,2100263,+10.48%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W,Houston (/ˈhjuːstən/ HEW-stən) is the most pop...
4,5,Phoenix,Arizona,1680992,1445632,+16.28%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°W,Phoenix (/ˈfiːnɪks/ FEE-niks; Navajo: Hoozdo; ...


In [18]:
#copy the data in topcities to ustopcities
ustopcities = topcities.copy()      

for i in range(0,len(ustopcities)):    #for each row
    for j in range(0,len(ustopcities.columns)):  #for each column
        ustopcities.iloc[i,j] = re.sub('\[.*?\]','',ustopcities.iloc[i,j])  #remove annotation symbols

TypeError: cannot use a string pattern on a bytes-like object

In [19]:
#find and store the index of the 'Population Change Percentage' column
pop = ustopcities.columns.get_loc("Population Change Percentage")

for i in range(0,len(ustopcities)):    #for each value
    ustopcities.iloc[i,pop] = re.sub('\[.*?\]|[+]|[%]','',ustopcities.iloc[i,pop])  #remove redundant data and symbols
    ustopcities.iloc[i,pop] = re.sub('−','-',ustopcities.iloc[i,pop]) #change the negative symbol for further transformation
    if ustopcities.iloc[i,pop] != "": #if the value is not null
        ustopcities.iloc[i,pop] = float(ustopcities.iloc[i,pop])  #transform the value into float format

KeyError: 'Population Change Percentage'

In [20]:
 #create a list to contain the column names
areas = ['Land Area2016','Land Area Latest', 'Water Area Latest', 'Metro Area Latest', 'Urban Area Latest']

for area in areas:  #for each area column
    index = ustopcities.columns.get_loc(area)  #find and store the index of the column
    for i in range(0,len(ustopcities)):    #for each value
        ustopcities.iloc[i,index] = re.sub('sq|mi|\(.*?\).*?$|\[.*?\]|,|[A-Za-z]+|:|–|\\xa0.*?$','',ustopcities.iloc[i,index])  #remove redundant data and symbols
        if 'km2' in ustopcities.iloc[i,index]:   #if the remaining data's unit is in km2
            ustopcities.iloc[i,index] = re.sub('km2','',ustopcities.iloc[i,index])  #remove the text km2
            ustopcities.iloc[i,index] = round(float(ustopcities.iloc[i,index])*0.386102,2)  #transform the data into square miles
        if ustopcities.iloc[i,index] != "" and area != 'Urban Area Latest':    #for strings that are not empty in the first three columns
            ustopcities.iloc[i,index] = float(ustopcities.iloc[i,index])        #transform the value into float format

KeyError: 'Land Area2016'

In [21]:
#find and store the index of the 'Population Density2016' column
den = ustopcities.columns.get_loc("Population Density2016") 

for i in range(0,len(ustopcities)):    #for each value
    ustopcities.iloc[i,den] = re.sub('\/sq|mi|,','',ustopcities.iloc[i,den])  #remove redundant data and symbols
    ustopcities.iloc[i,den] = int(ustopcities.iloc[i,den])  #transform the value into integer format

KeyError: 'Population Density2016'

In [22]:
#find and store the index of the 'Location' column
loc = ustopcities.columns.get_loc("Location") 

for i in range(0,len(ustopcities)):    #for each value
    ustopcities.iloc[i,loc] = re.sub('\/.*?$','',ustopcities.iloc[i,loc]) #remove redundant and duplicated data

In [23]:
#find and store the index of the 'Elevation' column
ele = ustopcities.columns.get_loc("Elevation") 

for i in range(0,len(ustopcities)):    #for each value
    ustopcities.iloc[i,ele] = re.sub('\(.*?\).*?$|ft|,|\[.*?\]','',ustopcities.iloc[i,ele])  #remove redundant data and symbols
    ustopcities.iloc[i,ele] = re.sub('\sto\s|-|–|\s-\s',',',ustopcities.iloc[i,ele]) #change the seperation symbol to ','
    if 'm' in ustopcities.iloc[i,ele]:   #if the remaining data's unit is in m
        ustopcities.iloc[i,ele] = re.sub('m','',ustopcities.iloc[i,ele])  #remove the text m
        ustopcities.iloc[i,ele] = round(float(ustopcities.iloc[i,ele])*3.28084,0)  #transform the data into feet
    if ustopcities.iloc[i,ele] != "": #if the value is not null
        try:
            ustopcities.iloc[i,ele] = int(round(float(ustopcities.iloc[i,ele]),1))  #transform the value into integer
        except:
            lst = ustopcities.iloc[i,ele].split(',') #for values that contain a range of evelation
            lst[0] = lst[0].replace('−','-')
            lst[1] = lst[1].replace('\xa0 ','')
            ustopcities.iloc[i,ele] = int((float(lst[0])+float(lst[1]))/2) #calculate the average evelation and transform to integer
            

KeyError: 'Elevation'

In [24]:
#find and store the index of the column Incorporated
incop = ustopcities.columns.get_loc('Incorporated')     
dlist = []    #build a new list to store the index of the rows that have raised exceptions

for i in range(0,len(ustopcities)):     #for each row in the data frame
    ustopcities.iloc[i,incop] = re.sub('\[.*?\]|\s\(.*?\)|\(.*?$|,\s[a-z].*$|\s[a-z].*$|;\s[1-9].*?$|:[1-9].*?$','',ustopcities.iloc[i,incop]) #remove meaningless symbols
    try:     #use try-except block to catch exceptions
        ustopcities.iloc[i,incop] = pd.to_datetime(ustopcities.iloc[i,incop])  #transform the data into datetime format
    except:
        print(i,ustopcities.iloc[i,incop])  #if there appears any exceptions, print out the row index and value
        dlist.append(i)  #add the row indexes into the list

KeyError: 'Incorporated'

In [25]:
 #for each row in the list
for i in dlist: 
    dlength = len(ustopcities.iloc[i,incop])   #calculate the length of the string in the row
    ustopcities.iloc[i,incop] = pd.to_datetime(ustopcities.iloc[i,incop][dlength-4:],errors='coerce')  #select the last 4 digitd and transform it into datetime format, remaining errors are coerced

NameError: name 'dlist' is not defined

In [26]:
#inspect the summary of ustopcities
ustopcities.describe()   

,2019rank,2019estimate,2010Census
count,317.000000,3.170000e+02,3.170000e+02
mean,158.996845,3.011608e+05,2.776344e+05
std,91.651013,5.813693e+05,5.605536e+05
min,1.000000,1.001450e+05,0.000000e+00
25%,80.000000,1.189270e+05,1.095650e+05
50%,159.000000,1.594280e+05,1.504410e+05
75%,238.000000,2.620750e+05,2.361230e+05
max,317.000000,8.336817e+06,8.175133e+06


In [27]:
ustopcities.to_csv('E:\TopUSCities_Yirou.csv',encoding='utf-8', index=False)